In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN

In [ ]:
from collections import Counter

In [ ]:
from hyperopt import hp

In [ ]:
import spacy

In [ ]:
from hyperopt import fmin, tpe, Trials

In [ ]:
import json

In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans

In [ ]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')






[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
%cd /content/drive/MyDrive/MS Hackathon


/content/drive/MyDrive/MS Hackathon


In [ ]:
def calculate_similarity_matrix(comment_vectors):
    return cosine_similarity(comment_vectors)

In [ ]:
def clean_text(text):
#     remove leading and trailing spaces
    text=text.strip()
    # Remove links
    text = re.sub(r'http\S+', '', text)
    # Remove non-UTF-8/English characters
    text = text.encode('ascii', 'ignore').decode('utf-8')
#     remove numeric vals
    text=' '.join([word for word in text.split() if not word.isdigit()])
     # Tokenize the text
    tokens = nltk.word_tokenize(text)
    # Remove punctuation and stop words, and lemmatize tokens
    lemmatizer = WordNetLemmatizer()
    cleaned_tokens = [lemmatizer.lemmatize(token.lower()) for token in tokens if token.lower() not in string.punctuation and token.lower() not in stopwords.words('english')]
    # Join the cleaned and lemmatized tokens into a string

    cleaned_text = ' '.join(cleaned_tokens)

    return cleaned_text

In [ ]:
!pip install -U sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
with open("1.json", "r") as file:
    data = json.load(file)


In [ ]:
raw_data=set() #the raw data is the links and the desc in a set of strings
tags={}
for obj in data:
    for results in obj['items']:
        title_sents=nltk.sent_tokenize(results["snippet"]["title"])
        for sent in title_sents:
            raw_data.add(sent)
        descs=results["snippet"]["description"]
        sentences = nltk.sent_tokenize(descs)
        for sent in sentences:
            raw_data.add(sent)

        if "tags" in results["snippet"].keys():
          for tag in results["snippet"]["tags"]:
            if tag not in tags.keys():
              tags[tag]=1
            else:
              tags[tag]+=1


In [ ]:
tags

In [ ]:
import pickle

In [ ]:
with open('1tags.pkl', 'wb') as f:
    pickle.dump(tags, f)

In [ ]:
len(raw_data)

In [ ]:
for sent in raw_data:
    print(sent)

In [ ]:
cleaned_data=set()
for sent in raw_data:
    cleaned_data.add(clean_text(sent))


In [ ]:
len(cleaned_data)

In [ ]:
for sent in cleaned_data:
    print(sent)
    print("---------------")

In [ ]:
data=list(cleaned_data)

In [ ]:
data[1]

In [ ]:
data[2]

In [ ]:
embeddings = model.encode(data)

In [ ]:
cs_mat=calculate_similarity_matrix(embeddings)

In [ ]:
cs_mat[1][2]

In [ ]:
space = {
    'eps': hp.uniform('eps', 0.1, 2.0),
    'min_samples': hp.quniform('min_samples', 1, 10, 1)
}

In [ ]:
def objective(params):
    eps = params['eps']
    min_samples = int(params['min_samples'])
    if len(cs_mat) < min_samples:
        return -1.0  # Return a low score if the number of samples is too small

    dbscan = DBSCAN(eps=eps, min_samples=min_samples, metric='euclidean')
    labels = dbscan.fit_predict(cs_mat)

    # Check if there are more than one unique label (clusters)
    unique_labels = len(set(labels))
    if unique_labels > 1:
        score = silhouette_score(cs_mat, labels, metric='euclidean')
    else:
        # If only one or zero clusters, return a very low score
        score = -1.0

    return -score  # Hyperopt minimizes, so we negate the score to maximize it

In [ ]:
trials = Trials()
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=50, trials=trials)

best_eps = best['eps']
best_min_samples = int(best['min_samples'])

NameError: ignored

In [ ]:
best_eps

NameError: ignored

In [ ]:
best_min_samples

NameError: ignored

In [ ]:
best_dbscan = DBSCAN(eps=best_eps, min_samples=best_min_samples, metric='euclidean')
best_labels = best_dbscan.fit_predict(cs_mat)


In [ ]:
best_labels

array([-1, -1, -1,  0, -1, -1, -1,  0, -1, -1, -1,  0,  0,  0,  0,  0,  0,
       -1, -1,  0, -1,  0, -1,  0, -1, -1, -1, -1, -1, -1,  0, -1, -1, -1,
       -1, -1, -1, -1,  0, -1, -1, -1,  0, -1, -1, -1, -1, -1, -1, -1, -1,
        0, -1,  0, -1, -1, -1, -1, -1, -1, -1,  0,  0, -1, -1,  0, -1,  0,
       -1,  0,  0, -1, -1, -1, -1, -1,  0,  0,  0, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1,  0, -1, -1, -1,  0, -1, -1,  0,  0, -1,
       -1, -1,  0, -1,  0, -1,  0, -1, -1,  0,  0,  0,  0, -1, -1, -1,  0,
       -1, -1, -1,  0, -1, -1, -1,  0, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1,  0,  0,  0, -1,  0, -1, -1, -1, -1, -1, -1,  0, -1,  0, -1,
        0, -1, -1, -1, -1,  0,  0,  0, -1, -1,  0, -1,  0,  0,  0, -1, -1,
        0,  0, -1, -1,  0, -1, -1,  0,  0, -1, -1,  0,  0,  0, -1,  0,  0,
       -1, -1,  0,  0,  0, -1,  0, -1, -1, -1,  0, -1,  0, -1, -1, -1, -1,
        0,  0,  0,  0, -1, -1, -1, -1,  0, -1,  0,  0, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1

In [ ]:
groups={x:[] for x in range(-1,best_labels.max() + 1)}
for i in range(len(best_labels)):
  groups[best_labels[i]].append(data[i])


In [ ]:
groups

{-1: ['',
  '05:10 heating air conditioning sustainable sign fortnightly newsletter climate change find climate change coverage use renewable energy accelerating read first big energy shock green era bottleneck could constrain emission cut government spurred rise solar power read new solar cell extract energy sunshine watch film cool warming world nuclear power must well regulated ditched smaller reactor make nuclear power economic',
  'thank much patron support',
  'ocean mechanical energy us ebb flow tide generate energy created earth rotation gravity moon',
  'film supported infosys 00:00 energy need become sustainable 00:33 much energy come renewables',
  'maybe come decision',
  'channel short clip various podcasts shared',
  'look big small system need change',
  'game interactive story video five language english spanish portuguese italian french',
  'wind turbine built windy',
  'lot',
  'without video would possible',
  "twitter instagram facebook linkedin tiktok ted talk chan

In [ ]:
groups[0]

['let see come decision turning waste energy worth',
 'type renewable energy',
 "many advantage renewable energy environmentally friendly replenish naturally safer n't generate toxic waste",
 'video child find sunlight energy eolic energy tidal wave energy hydraulic energy geothermal energy',
 'green solar energy really',
 'come cost clean energy bound beat fossil fuel say technologist ramez naam',
 "solar panel popping world 's easy see provide clean energy falling price",
 'china plan win future energy',
 'making clean energy isnt enough also move',
 'fix clean energy storage problem',
 'renewable energy save planet',
 'together address challenge increasing energy access billion also building infrastructure efficient sustainable',
 'sustainable energy',
 'provocative talk time magazine hero environment energy expert michael shellenberger explains solar wind farm require much land mining energy production alternative path saving climate natural environment',
 'energy central powering 

In [ ]:
def compute_IDF(documents):
    word_count = Counter()
    for doc in documents:
        if 'drops(players' in doc:
            print(doc)
        word_set = set(doc.split())
        word_count.update(word_set)
    # print(word_count)
    total = sum(word_count.values())
    word_IDF = {k : round(( np.log2(total / v ) )) for k, v in word_count.items() }
    return word_IDF



In [ ]:
word_IDF = compute_IDF(data)
word_IDF

{'accelerating': 11,
 'read': 10,
 'government': 11,
 'cool': 12,
 'shock': 12,
 'find': 9,
 '05:10': 12,
 'climate': 8,
 'regulated': 12,
 'energy': 5,
 'change': 9,
 'bottleneck': 11,
 'smaller': 12,
 'must': 11,
 'make': 8,
 'heating': 12,
 'spurred': 12,
 'constrain': 12,
 'new': 8,
 'world': 7,
 'sign': 10,
 'film': 10,
 'conditioning': 12,
 'big': 11,
 'green': 8,
 'era': 12,
 'first': 9,
 'could': 9,
 'emission': 9,
 'rise': 12,
 'use': 8,
 'extract': 11,
 'solar': 7,
 'warming': 10,
 'newsletter': 10,
 'nuclear': 10,
 'well': 9,
 'cell': 11,
 'ditched': 12,
 'air': 10,
 'sustainable': 7,
 'coverage': 10,
 'watch': 8,
 'sunshine': 12,
 'fortnightly': 12,
 'power': 7,
 'economic': 12,
 'cut': 11,
 'renewable': 6,
 'reactor': 11,
 'support': 9,
 'thank': 10,
 'much': 8,
 'patron': 11,
 'worth': 12,
 'see': 9,
 'come': 9,
 'waste': 9,
 'let': 11,
 'decision': 11,
 'turning': 10,
 'gravity': 12,
 'rotation': 12,
 'ocean': 10,
 'generate': 11,
 'flow': 11,
 'tide': 12,
 'ebb': 12,
 '

In [ ]:
def most_common(lst, n_words ):
    counter = Counter(lst)
    for k in list(counter):
        if counter[k] == 1:
            pass
        else:
            if k in word_IDF.keys():
              counter[k] *= word_IDF[k]

    return counter.most_common(n_words)

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
def extract_labels(category_doc, print_word_counts = False):
    '''
    Argument:
        category_docs: list of documents, all from the same category or clustering. category_docs is a list of strings like below
        ['fun game small amount time unless lifer..........', ....  'steem pile graphic' ]

        print_word_counts: bool, True will print word counts of each type in this category
    '''

    verbs = []
    dobjs = []
    nouns = []
    adjs = []

    verb = ''
    dobj = ''
    noun1 = ''
    noun2 = ''

    for i in range(len(category_doc)):
        doc = nlp(category_doc[i])
        for token in doc:
            if (token.is_stop == False ) and (len(str(token).strip()) >0 ):
                if token.pos_ == 'VERB':
                    verbs.extend([token.lemma_.lower()])

                elif token.dep_ == 'dobj':
                    dobjs.extend([token.lemma_.lower()])

                elif token.pos_ == 'NOUN':
                    nouns.extend([token.lemma_.lower()])

                elif token.pos_ == 'ADJ':
                    adjs.extend([token.lemma_.lower()])


    # print('most_common(verbs, 1) ', most_common(verbs, 1))
    # [('m', 27)]

    if print_word_counts:
        for word_lst in [verbs, dobjs, nouns, adjs]:
            counter = Counter(word_lst)
            print(counter)

    if len(verbs) > 0:
        verb = most_common(verbs, 1)[0][0]

    if len(dobjs) > 0:
        dobj = most_common(dobjs, 1)[0][0]

    if len(nouns) > 0:
        noun1 = most_common(nouns, 1)[0][0]

    if len(set(nouns)) > 1:
        noun2 = most_common(nouns, 2)[1][0]


    # concatenate the most common verb-dobj-noun1-noun2 (if they exist)
    label_words = [verb, dobj]

    for word in [noun1, noun2]:
        if word not in label_words:
            label_words.append(word)

    if '' in label_words:
        label_words.remove('')

    label = '_'.join(label_words)

    return label



In [ ]:
positive_label_count={x:((len(groups[x])/len(data))*100) for x in groups.keys()}

In [ ]:
positive_label_count

{-1: 67.32673267326733, 0: 32.67326732673268}

In [ ]:
series=pd.Series(positive_label_count)

In [ ]:
series.index=[extract_labels(groups[x]) for x in groups.keys()]
series

learn_u_energy_video    67.326733
lead_energy_source      32.673267
dtype: float64

In [ ]:
series.to_csv("1topiclabels.csv")

# Mining

In [ ]:
for topic in range(10):
    print(f"Doing Topic no: {topic}")
    with open(f"{topic}.json", "r") as file:
        data = json.load(file)
    raw_data = set()  # the raw data is the links and the desc in a set of strings
    tags = {}
    for obj in data:
        for results in obj['items']:
            title_sents = nltk.sent_tokenize(results["snippet"]["title"])
            for sent in title_sents:
                raw_data.add(sent)

            # Handle missing description
            try:
                descs = results["snippet"]["description"]
                sentences = nltk.sent_tokenize(descs)
                for sent in sentences:
                    raw_data.add(sent)
            except KeyError:
                descs = ""  # Set an empty string if "description" is missing

            # Handle missing tags
            try:
                for tag in results["snippet"]["tags"]:
                    if tag not in tags.keys():
                        tags[tag] = 1
                    else:
                        tags[tag] += 1
            except KeyError:
                pass

    with open(f"{topic}tags.pkl", "wb") as f:
        pickle.dump(tags, f)
        print(f"Created {topic}tags.pkl")



    cleaned_data=set()
    for sent in raw_data:
        cleaned_data.add(clean_text(sent))

    data=list(cleaned_data)
    embeddings=model.encode(data)
    cs_mat=calculate_similarity_matrix(embeddings)
    space = {
        'eps': hp.uniform('eps', 0.1, 2.0),
        'min_samples': hp.quniform('min_samples', 1, 10, 1)
    }
    trails=Trials()
    best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=50, trials=trials)

    best_eps = best['eps']
    best_min_samples = int(best['min_samples'])
    best_dbscan = DBSCAN(eps=best_eps, min_samples=best_min_samples, metric='euclidean')
    best_labels = best_dbscan.fit_predict(cs_mat)
    groups={x:[] for x in range(-1,best_labels.max() + 1)}
    for i in range(len(best_labels)):
      groups[best_labels[i]].append(data[i])

    word_IDF = compute_IDF(data)
    positive_label_count={x:((len(groups[x])/len(data))*100) for x in groups.keys()}
    series=pd.Series(positive_label_count)
    series.index=[extract_labels(groups[x]) for x in groups.keys()]
    print(series)
    series.to_csv(f"{topic}topicmodel.csv")
    print(f"Created Topic Model: {topic}")

Doing Topic no: 0
Created 0tags.pkl
100%|██████████| 50/50 [00:00<?, ?trial/s, best loss=?]
find_energy_video            87.286528
type_energy_source            9.867173
replenish_resource_energy     2.846300
dtype: float64
Created Topic Model: 0
Doing Topic no: 1
Created 1tags.pkl
100%|██████████| 50/50 [00:00<?, ?trial/s, best loss=?]
learn_u_energy_video    67.326733
lead_energy_source      32.673267
dtype: float64
Created Topic Model: 1
Doing Topic no: 2
Created 2tags.pkl
100%|██████████| 50/50 [00:00<?, ?trial/s, best loss=?]
find_video_energy        55.773956
lead_energy_wind         42.260442
follow_twitter_seeker     1.965602
dtype: float64
Created Topic Model: 2
Doing Topic no: 3
Created 3tags.pkl
100%|██████████| 50/50 [00:00<?, ?trial/s, best loss=?]
watch_energy_battery    76.171875
know_panel_energy       23.828125
dtype: float64
Created Topic Model: 3
Doing Topic no: 4
Created 4tags.pkl
100%|██████████| 50/50 [00:00<?, ?trial/s, best loss=?]
watch_rate_battery_video      